In [1]:
import pandas as pd
import numpy as np

import re

from py2neo import authenticate, Graph, Node, Relationship


import os
import csv
import pickle

from time import sleep
from timeit import default_timer as timer
from datetime import datetime

from IPython.display import display, HTML

# custom general helper functions for this project
import custom_utils as cu
import importlib


In [2]:
from collections import defaultdict

In [3]:
# reload imports as needed
importlib.reload(cu);

In [4]:
# set up Pandas options
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 3)
pd.options.display.float_format = '{:.2f}'.format

In [5]:
pd.options.display.max_colwidth = 100

In [6]:
# unpickle
with open("pickles/en_1218_louvain_communities_for_NLP.pkl", 'rb') as picklefile: 
    louvain_communities_for_NLP = pickle.load(picklefile)

louvain_communities_for_NLP.head(20)

,external_search_traffic,link_in_traffic,louvain_community,search_in_traffic,title
0,4576854.00,1108189.00,3,5630.00,George_H._W._Bush
1,3538068.00,639353.00,4,6451.00,Jason_Momoa
2,3475113.00,223635.00,9,23563.00,2.0_(film)
3,3251996.00,682992.00,4,10416.00,Bird_Box_(film)
4,3020671.00,31170.00,1,nan,Main_Page
5,2634665.00,408421.00,4,34309.00,Aquaman_(film)
6,2328884.00,200893.00,4,192.00,Bird_Box
7,2231176.00,575481.00,3,3945.00,Priyanka_Chopra
8,2226602.00,117115.00,5,958.00,List_of_most-disliked_YouTube_videos
9,2050628.00,336621.00,5,4161.00,Freddie_Mercury


In [37]:
louvain_communities_for_NLP[louvain_communities_for_NLP.external_search_traffic.isnull()][:5]

,external_search_traffic,link_in_traffic,louvain_community,search_in_traffic,title
2401006,nan,12.00,2,nan,Orange-brown_Atlantic_tree-rat
2401007,nan,16.00,4,nan,Eilis_Kirwan
2401008,nan,10.00,1,nan,Emile_Waxweiler
2401009,nan,213.00,7,nan,Sun_King_(disambiguation)
2401010,nan,29.00,3,nan,Ahmet_Lepenica


In [75]:
community_deepWiki_stats = louvain_communities_for_NLP[louvain_communities_for_NLP.external_search_traffic.isnull()] \
    .groupby("louvain_community")[["title"]].count()
    
community_deepWiki_stats.columns = ["deep_wiki_articles_count"]

community_deepWiki_stats[:10]

,deep_wiki_articles_count
louvain_community,
0,3311
1,16439
2,28617
3,71293
4,22471
5,27513
6,5251
7,37641
8,10764


In [36]:
# unpickle
with open("pickles/en_1218_louvain_communities.pkl", 'rb') as picklefile: 
    louvain_communities = pickle.load(picklefile)

louvain_communities.head(5)

,articles_count,external_search_traffic,link_edges_count,link_traffic,louvain_community,search_edges_count,search_traffic,total_visits,avg_external_search_traffic,avg_link_traffic_per_edge,avg_visits_per_article,link_network_density,link_network_density_delta
0,507642,535227964,3385462,329625267,3,102124,4188154,1124655652,1054.34,97.36,2215.45,0.00,5.67
1,302120,288453340,2037310,140416238,7,41383,1257742,592415339,954.76,68.92,1960.86,0.00,5.74
2,267541,626988532,2540296,461475849,4,98857,3938984,1350876078,2343.52,181.66,5049.23,0.00,8.50
3,233396,150495954,1209989,84128861,10,25894,932688,322074785,644.81,69.53,1379.95,0.00,4.18
4,223864,219874886,1327054,151652674,5,32332,930331,468611692,982.18,114.28,2093.29,0.00,4.93


In [73]:
louvain_communities_size = louvain_communities.set_index("louvain_community").articles_count
louvain_communities_size.head()

louvain_community
3     507642
7     302120
4     267541
10    233396
5     223864
Name: articles_count, dtype: int64

In [77]:
community_deepWiki_stats = community_deepWiki_stats.join(louvain_communities_size)
community_deepWiki_stats.head()

,deep_wiki_articles_count,articles_count
louvain_community,,
0,3311,29397
1,16439,218001
2,28617,209065
3,71293,507642
4,22471,267541


In [78]:
community_deepWiki_stats["deep_wiki_articles_proportion"] = \
    community_deepWiki_stats.deep_wiki_articles_count / community_deepWiki_stats.articles_count
    
community_deepWiki_stats.head()

,deep_wiki_articles_count,articles_count,deep_wiki_articles_proportion
louvain_community,,,
0,3311,29397,0.11
1,16439,218001,0.08
2,28617,209065,0.14
3,71293,507642,0.14
4,22471,267541,0.08


In [83]:
# unpickle
with open("pickles/en_1218_louvain_community_topics_dict.pkl", 'rb') as picklefile: 
    community_topics_dict = pickle.load(picklefile)

community_topics_dict[22]["topic_words"][:5]

['grand', 'prix', 'championship', 'seri', 'engin']

In [84]:
community_ids = community_deepWiki_stats.index.values

for i in community_ids:
    
    #print(community_deepWiki_stats.at[i, "deep_wiki_articles_count"] )
    
    community_deepWiki_stats.at[i, "topic_words"] = " ".join(community_topics_dict[i]["topic_words"])
       

In [85]:
community_deepWiki_stats[:10]

,deep_wiki_articles_count,articles_count,deep_wiki_articles_proportion,topic_words
louvain_community,,,,
0,3311,29397,0.11,station railway class line rail metro list park train transport british london south road tram c...
1,16439,218001,0.08,list disambigu softwar power station number engin space unit program network model theorem group...
2,28617,209065,0.14,list disambigu syndrom nation park diseas food black acid hospit state red white cell medic dog ...
3,71293,507642,0.14,list unit state footbal elect nation th constitu parti battl john film st district war cup disam...
4,22471,267541,0.08,film list seri tv actor comic season award disambigu episod school john charact novel man star l...
5,27513,223864,0.12,album song band list discographi love music live musician record tour disambigu ep best world aw...
6,5251,81807,0.06,univers school list colleg bank institut compani state intern educ group law technolog scienc un...
7,37641,302120,0.12,list disambigu church al john novel saint languag histori st film battl ii cathol roman book wil...
8,10764,73172,0.15,airport air uss list class hms airlin intern squadron flight submarin forc aircraft ship th avia...


In [89]:
community_deepWiki_stats[community_deepWiki_stats.articles_count >= 100].sort_values("deep_wiki_articles_proportion", ascending=False)[:50]

,deep_wiki_articles_count,articles_count,deep_wiki_articles_proportion,topic_words
louvain_community,,,,
59,147,159,0.92,kingdom list locat unit templecomb zeal st west cornwal new aa ak old north monachorum zouch eas...
196,62,111,0.56,art tabl year archaeolog architectur
136,44,120,0.37,music paul rapin comet hanov treati peac vienna del francesco giudic
84,76,278,0.27,orchestra symphoni philharmon state youth dispatch new chamber san album fiedler pop unit sander...
12,13319,49144,0.27,olymp championship summer men world women game athlet metr open winter cup singl list swim tenni...
160,29,111,0.26,cyprus elect cypriot legisl presidenti parti democrat list movement newspap european anastasiad ...
19,5206,22194,0.23,footbal leagu cup nation malaysia district stadium championship fc list team asian afc station f...
44,717,3171,0.23,snooker dart world championship player master open pdc cup uk bdo leagu pool grand rank season t...
76,35,166,0.21,bc decad mentuemhat
